In [1]:
import keras
import numpy as np 
from tqdm import tqdm 
import tensorflow as tf

from glob import glob
import pathlib

from keras.preprocessing.image import img_to_array, load_img



from PIL import Image

import shutil
import matplotlib.pyplot as plt
import numpy as np


import os
import subprocess
from IPython.display import FileLink, display

In [2]:
images = glob(r'../input/aisegmentcom-matting-human-datasets/clip_img/*/*/*')
masks = glob(r'../input/aisegmentcom-matting-human-datasets/matting/*/*/*')

## Fill masks

In [4]:

def fill_mask(mask):
    # See for mask as vector string and fill with one this string from first non zero value to the last non zero value
    m = plt.imread(mask)
    m = m[:,:,0]
    m = (m[:,:]>0).astype(np.float16)
    
    for i, vector in enumerate(m):
        Z = np.zeros_like(vector)
        O = np.ones_like(vector)
        one_indexs = []
        
        for i2, el in enumerate(vector):
            if el > 0:
                one_indexs.append(i2)
        if one_indexs:
            Z[one_indexs[0]:one_indexs[-1]] = O[one_indexs[0]:one_indexs[-1]]
            m[i,:] = Z
    return m
    
    

## Copy masks

In [5]:

def copy_masks(masks):
    new_dir = os.path.join('/kaggle/working/', 'masks')
    try:
        os.mkdir(new_dir)    
    except FileExistsError:
        pass

    for masks_path in tqdm(masks, total = len(masks)):
        mask = fill_mask(masks_path)
        #mask = mask.astype(bool)
        path = masks_path.split('/')
        new_dir = os.path.join('/kaggle/working/masks',path[-3], path[-2])
        try:
            os.makedirs(new_dir) 
        except FileExistsError:
            pass
        plt.imsave(os.path.join(new_dir, path[-1][:-4]+'.png'), mask)
 


        



In [6]:
copy_masks(masks)

100%|██████████| 34426/34426 [10:28:49<00:00,  1.10s/it] 


In [7]:


def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

In [8]:
download_file('/kaggle/working/', 'masks_p1')

/kaggle/working/masks_p1.zip

In [11]:
shutil.make_archive('masks', 'zip', '/kaggle/working/masks/')

'/kaggle/working/masks.zip'